In [36]:
import os
import pandas as pd
import ssl
import json

from kafka import KafkaProducer
import time

ssl._create_default_https_context = ssl._create_unverified_context

In [35]:
KAFKA_BOOTSTRAP_SERVERS='localhost:9092'
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [4]:
url = "/home/data_000019.txt"
df = pd.read_csv(url)

In [27]:
jj = df.loc[].to_json()

In [41]:

for i in range(20):
    producer.send('my_awesome_topic', df.loc[i].to_json().encode('utf-8'))
    producer.flush()
    time.sleep(1)


KeyboardInterrupt: 

In [37]:
for i in range(0,81):
    i = str(i).zfill(2)
    url = f"https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_0000{i}.txt"
    df = pd.read_csv(url)
    for j in range(0,df.shape[0]//1000):
        producer.send('a_partitioned_topic', json.dumps(msg).encode('utf-8'))
        producer.flush()
        time.sleep(0.25)
        

KeyboardInterrupt: 

In [13]:
producer.send('topic_stream', b'message 1')
producer.flush()

# Loro

In [15]:
import socket
import json
import time
import random

first_names=('John','Andy','Joe','Alice')
last_names=('Johnson','Smith','Jones', 'Millers')

# while 1:
for i in range(20):
    msg = {
        'name': random.choice(first_names),
        'surname': random.choice(last_names),
        'amount': '{:.2f}'.format(random.random()*1000),
        'delta_t': '{:.2f}'.format(random.random()*10),
        'flag': random.choices([0,1], weights=[0.8, 0.2])[0]
    }
    producer.send('topic_stream', json.dumps(msg).encode('utf-8'))
    producer.flush()
    time.sleep(0.25)